In [ ]:
%cd ../..
# %pip install .
import time
from os import environ
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# point1 = OrientedPoint ahead of ego by Range(0, 40)
# Car at (point1 offset by Range(-1, 1) @ 0),
#     facing Range(-15, 15) deg relative to roadDirection



In [ ]:
if environ["AP_PORT"] is None:
    environ["AP_PORT"] = str(input('port'))

In [ ]:
from show_images import show_images
%matplotlib inline

# data_dir =  "data/nuscenes/experiment_data"
# data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/experiment_data'
data_dir = '/Users/chanwutk/Documents/experiment_data'

In [ ]:
#### First part of the query ####
from apperception.world import empty_world
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.database import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()


name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
# world = world.predicate(lambda obj: obj.object_type == 'vehicle')
def pred(obj1, cam):
    return (
        F.like(obj1.object_type, 'vehicle%') and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) > -15 and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) < 15 and
        F.view_angle(obj1, cam.ego, cam.timestamp) < 67.5 and
        F.distance(cam.ego, obj1, cam.timestamp) < 40 and
        F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp) > -15 and 
        F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp) < 15 and
        F.ahead(obj1, cam.ego, cam.timestamp)
    )
world = world.filter("lambda obj1, cam: " +
        "F.like(obj1.object_type, 'vehicle%') and " +
        "F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp), -15, 15) and " +
        "F.view_angle(obj1, cam.ego, cam.timestamp) < 70 / 2 and " +
        "F.distance(cam.ego, obj1, cam.timestamp) < 40 and " +
        "F.angle_between(F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp), -15, 15) and " +
        "F.ahead(obj1, cam.ego, cam.timestamp)")

start = time.time()
# keys = world.get_traj_key()
id_time_camId_filename_1 = world.get_id_time_camId_filename(num_joined_tables=1)

end = time.time()
print(format(end-start))

In [ ]:
resultImages_1 = dict() # maping from image -> (itemId, timestamp, camId) that it was found at
for result in id_time_camId_filename_1:
    itemId, timestamp, camId, filename = result
    filename = filename.split("/")[-1] # use split so that prefix path is not included in filename
    resultImages_1[filename] = (itemId, timestamp, camId)
    if filename == 'n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920162404.jpg':
        print("exists")

In [ ]:
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# oppositeCar = Car offset by (Range(-10, -1), Range(0, 50)),
#     facing Range(140, 180) deg relative to ego.heading

# point2 = OrientedPoint ahead of oppositeCar by Range(0, 40)
# Car at (point2 offset by Range(-1, 1) @ 0),
#     facing Range(-15, 15) deg relative to roadDirection

In [ ]:
### Second part of the query ###
from apperception.world import empty_world

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F

world = world.filter("lambda opposite_car, car2, cam: " +
        "F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp), -15, 15) and " +
        "F.convert_camera(opposite_car, cam.ego, cam.timestamp) > [-10, 0] and " +
        "F.convert_camera(opposite_car, cam.ego, cam.timestamp) < [-1, 50] and " +
        "F.angle_between(F.facing_relative(opposite_car, cam.ego, cam.timestamp), 140, 180) and " +
        "F.like(car2.object_type, 'vehicle%') and F.like(opposite_car.object_type, 'vehicle%') and " +
        "opposite_car.itemId != car2.itemId and " +
        "F.distance(opposite_car, car2, cam.timestamp) < 40 and " +
        "F.angle_between(F.facing_relative(car2, F.road_direction(car2.traj, cam.timestamp, cam.ego), cam.timestamp), -15, 15) and " +
        "F.ahead(car2, opposite_car, cam.timestamp)")

start = time.time()
# keys = world.get_traj_key()
id_time_camId_filename_2 = world.get_id_time_camId_filename(num_joined_tables=2)
# print([e[4] for e in id_time_camId_filename_2])
end = time.time()
print(format(end-start))

In [ ]:
resultImages_2 = dict() # maping from image -> (itemId, timestamp, camId) that it was found at
for result in id_time_camId_filename_2:
    itemId_1, itemId_2, timestamp, camId, filename = result
    filename = filename.split("/")[-1] # use split so that prefix path is not included in filename
    resultImages_2[filename] = [(itemId_1, itemId_2, timestamp, camId)] if filename not in resultImages_2 else resultImages_2[filename]+[(itemId_1, itemId_2, timestamp, camId)]
print(resultImages_2['n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920162404.jpg'])

In [ ]:
len(resultImages_2)

In [ ]:
show_images(data_dir, list(resultImages_2.keys()), sample=10)

In [ ]:
expected_imgs = ["n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920162404.jpg", "n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920662404.jpg"]
show_images(data_dir, expected_imgs)

In [ ]:
### Getting info about the expected images
from apperception.database import database
for img in expected_imgs:
    prefix = "samples/CAM_FRONT/"
    query = f"""SELECT 
    \'Camera: \', Cameras.egoHeading, ST_X(Cameras.egoTranslation), ST_Y(Cameras.egoTranslation), ST_Z(Cameras.egoTranslation),
    \'Cars: \', table_0.itemId, getX(table_0.trajCentroids, Cameras.timestamp), getY(table_0.trajCentroids, Cameras.timestamp), 
    ST_Z(valueAtTimestamp(table_0.trajCentroids, Cameras.timestamp)), 
    \'Distance: \', distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp),
    \'CarHeading:\', valueAtTimestamp(table_0.itemHeadings, Cameras.timestamp),
    \'RoadDirection for car:\', roadDirection(table_0.trajCentroids, Cameras.timestamp, valueAtTimestamp(table_0.itemHeadings, Cameras.timestamp)::real),
    \'Ahead: \', ahead(table_0.trajCentroids, Cameras.egoTranslation, Cameras.egoHeading, Cameras.timestamp),
    \'Angle to ego: \', facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.timestamp) 
    FROM Item_General_Trajectory AS table_0, Cameras
                WHERE Cameras.filename = \'{prefix + img}\' AND table_0.cameraId = Cameras.cameraId 
                AND table_0.objectType LIKE 'vehicle%' 
                AND getX(table_0.trajCentroids, Cameras.timestamp) IS NOT NULL 
                AND ahead(table_0.trajCentroids, Cameras.egoTranslation, Cameras.egoHeading, Cameras.timestamp)
"""
    database.cursor.execute(query)
    result = database.cursor.fetchall()
    print(result)

In [ ]:
##test ahead#
from apperception.database import database
from datetime import datetime
for pair in resultImages_2['n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920162404.jpg']:
    itemId_1, itemId_2, timestamp, camId = pair
    query = f"""SELECT ahead(t2.trajCentroids,  t1.trajCentroids, t1.itemHeadings, cameras.timestamp),
                ST_AsText(valueAtTimestamp(t2.trajCentroids, cameras.timestamp)), 
                ST_AsText(valueAtTimestamp(t1.trajCentroids, cameras.timestamp)), 
                valueAtTimestamp(t1.itemHeadings, cameras.timestamp) 
            FROM Item_General_Trajectory AS t1, Item_General_Trajectory AS t2, Cameras
            WHERE t1.itemId = \'{itemId_1}\' AND t2.itemID = \'{itemId_2}\' AND Cameras.timestamp = \'{timestamp}\';"""
    database.cursor.execute(query)
    result = database.cursor.fetchall()
    print(result)

In [ ]:
##test ahead#
from apperception.database import database

query = f"""SELECT (1893.895627360433 - 1892.33194497499) * COS(PI() * (-120.12033333353695 + 90) / 180) + (1071.884001618687 - 1069.61352152532) * SIN(PI() * (-120.12033333353695 + 90) / 180),
         ABS(ST_X(convertCamera(st_point(1893.895627360433, 1071.884001618687), st_point(1892.33194497499, 1069.61352152532), -120.12033333353695)));"""
database.cursor.execute(query)
result = database.cursor.fetchall()
print(result)

In [ ]:
#### But for Scenic, all three cars should be in the scene ####
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# point1 = OrientedPoint ahead of ego by Range(0, 40)
# Car at (point1 offset by Range(-1, 1) @ 0),
#     facing Range(-15, 15) deg relative to roadDirection

# oppositeCar = Car offset by (Range(-10, -1), Range(0, 50)),
#     facing Range(140, 180) deg relative to ego.heading

# point2 = OrientedPoint ahead of oppositeCar by Range(0, 40)
# Car at (point2 offset by Range(-1, 1) @ 0),
#     facing Range(-15, 15) deg relative to roadDirection

from apperception.world import empty_world

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
world = world.filter("lambda car1, opposite_car, car2, cam: " +
        "F.like(car1.object_type, 'vehicle%') and " +
        "F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp), -15, 15) and " +
        "F.view_angle(car1, cam.ego, cam.timestamp) < 70 / 2 and " +
        "F.distance(cam.ego, car1, cam.timestamp) < 40 and " +
        "F.angle_between(F.facing_relative(car1, cam.ego, cam.timestamp), -15, 15) and " +
        "F.angle_between(F.facing_relative(car1, F.road_direction(car1.traj, cam.timestamp, cam.ego), cam.timestamp), -15, 15) and " +
        "F.ahead(car1, cam.ego, cam.timestamp) and " + 
        "F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp), -15, 15) and " +
        "F.convert_camera(opposite_car, cam.ego, cam.timestamp) > [-10, 0] and " +
        "F.convert_camera(opposite_car, cam.ego, cam.timestamp) < [-1, 50] and " +
        "F.angle_between(F.facing_relative(opposite_car, cam.ego, cam.timestamp), 140, 180) and " +
        "F.like(car2.object_type, 'vehicle%') and F.like(opposite_car.object_type, 'vehicle%') and " +
        "opposite_car.itemId != car2.itemId and car1.itemId != car2.itemId and car1.itemId != opposite_car.itemId and " +
        "F.distance(opposite_car, car2, cam.timestamp) < 40 and " +
        "F.angle_between(F.facing_relative(car2, F.road_direction(car2.traj, cam.timestamp, cam.ego), cam.timestamp), -15, 15) and " +
        "F.ahead(car2, opposite_car, cam.timestamp)")

start = time.time()
# keys = world.get_traj_key()
id_time_camId_filename_final = world.get_id_time_camId_filename(num_joined_tables=3)
end = time.time()
print(format(end-start))

In [ ]:
resultImages_final = dict() # maping from image -> (itemId, timestamp, camId) that it was found at
for result in id_time_camId_filename_final:
    car_1, oppositecar, car_2, timestamp, camId, filename = result
    filename = filename.split("/")[-1] # use split so that prefix path is not included in filename
    resultImages_final[filename] = [(car_1, oppositecar, car_2, timestamp, camId)] if filename not in resultImages_final else resultImages_final[filename]+[(itemId_1, itemId_2, timestamp, camId)]
print(resultImages_final['n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920162404.jpg'])

In [ ]:
len(resultImages_final)

In [ ]:
show_images(data_dir, list(resultImages_final.keys()), sample=10)

In [ ]:
missing = 0
missing_images = []
for expected_img in expected_imgs:
    if expected_img not in resultImages_final.keys():
        missing += 1
        missing_images.append(expected_img)
print("Percentage of expected results missing from query: ", missing, "/", len(expected_imgs), "=", 100 * missing / len(expected_imgs), "%")
show_images(data_dir, missing_images, sample=10)

In [ ]:
extra = 0
extra_images = {}
for img in resultImages_final.keys():
    if img not in expected_imgs:
        extra += 1
        extra_images[img] = resultImages_final[img]
print("Percentage of images in query but not in expected results: ", extra, "/", len(resultImages_final.keys()), "=", 100 * extra / len(resultImages_final.keys()), "%")
show_images(data_dir, extra_images, sample=10, seed=5)

In [ ]:
from apperception.database import database
from datetime import datetime
for pair in resultImages_final['n008-2018-08-21-11-53-44-0400__CAM_FRONT__1534867377412404.jpg']:
    car_1, oppositecar, car_2, timestamp, camId = pair
    query = f"""SELECT 
                distance(Cameras.egoTranslation, t1.trajCentroids, Cameras.timestamp),
                ahead(t2.trajCentroids,  t1.trajCentroids, t1.itemHeadings, cameras.timestamp),
                ST_AsText(valueAtTimestamp(t2.trajCentroids, cameras.timestamp)), 
                ST_AsText(valueAtTimestamp(t1.trajCentroids, cameras.timestamp)), 
                valueAtTimestamp(t1.itemHeadings, cameras.timestamp) 
            FROM Item_General_Trajectory AS t1, Item_General_Trajectory AS t2, Item_General_Trajectory AS t3, Cameras
            WHERE t1.itemId = \'{car_1}\' AND t2.itemID = \'{oppositecar}\' AND t3.itemId = \'{car_2}\'
            AND Cameras.timestamp = \'{timestamp}\';"""
    database.cursor.execute(query)
    result = database.cursor.fetchall()
    print(result)

In [ ]:
from apperception.database import database

query = f"""SELECT (1893.895627360433 - 1927.3625156512157) * COS(PI() * (59.95824 + 90) / 180) + (1071.884001618687 - 1053.3215015295318) * SIN(PI() * (59.95824 + 90) / 180),
         ST_AsText(convertCamera(st_point(1893.895627360433, 1071.884001618687), st_point(1927.3625156512157, 1053.3215015295318), 59.95824));"""
database.cursor.execute(query)
result = database.cursor.fetchall()
print(result)

In [ ]:
resultImages_final['n008-2018-08-21-11-53-44-0400__CAM_FRONT__1534867377412404.jpg']